# Day14_0 별첨: 이상탐지 (Anomaly Detection)

## 📚 학습 목표

**Part 1: 기초 - 이상탐지 개념 이해**
1. 이상탐지(Anomaly Detection)의 개념과 필요성 이해하기
2. 분류 문제와 이상탐지의 차이점 파악하기
3. 이상치(Anomaly)의 세 가지 유형 구분하기
4. Isolation Forest의 핵심 원리 이해하기

**Part 2: 심화 - 이상탐지 모델 활용**
1. Isolation Forest로 이상치 탐지 구현하기
2. One-Class SVM으로 경계 기반 탐지 수행하기
3. contamination 파라미터 최적화하기
4. 실무 데이터에 이상탐지 적용하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 이상탐지 | 비정상 패턴 감지 | 사기 탐지, 불량 탐지 |
| Isolation Forest | 고차원 이상치 탐지 | 신용카드 사기, 네트워크 침입 |
| One-Class SVM | 정상 데이터 경계 학습 | 제조 공정 이상, 의료 진단 |
| contamination | 이상치 비율 조절 | 도메인 지식 기반 임계값 설정 |

**분석가 관점**: 실제 비즈니스에서 이상치는 전체의 1% 미만입니다. 일반 분류 모델로는 극심한 불균형 문제가 발생하므로, 이상탐지 전용 알고리즘이 필수입니다!

---

## 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# sklearn 이상탐지 모델
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

# sklearn 유틸리티
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.datasets import make_blobs

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 랜덤 시드 고정
np.random.seed(42)

print("환경 설정 완료!")

## 데이터셋 준비: 합성 데이터

이상탐지 학습을 위해 정상 데이터와 이상치를 합성합니다.

In [ ]:
# 정상 데이터 생성 (군집 형태)
n_normal = 500
X_normal, _ = make_blobs(n_samples=n_normal, centers=1, cluster_std=1.0, random_state=42)

# 이상치 데이터 생성 (랜덤하게 퍼진 형태)
n_outliers = 25  # 전체의 약 5%
X_outliers = np.random.uniform(low=-6, high=6, size=(n_outliers, 2))

# 합치기
X = np.vstack([X_normal, X_outliers])
y_true = np.array([1] * n_normal + [-1] * n_outliers)  # 1: 정상, -1: 이상

# DataFrame 생성
df = pd.DataFrame(X, columns=['Feature_1', 'Feature_2'])
df['Label'] = ['정상' if y == 1 else '이상' for y in y_true]

print(f"전체 데이터: {len(df)}개")
print(f"정상 데이터: {n_normal}개 ({n_normal/len(df)*100:.1f}%)")
print(f"이상 데이터: {n_outliers}개 ({n_outliers/len(df)*100:.1f}%)")

In [ ]:
# 데이터 시각화
fig = px.scatter(df, x='Feature_1', y='Feature_2', color='Label',
                 title='정상 데이터 vs 이상 데이터',
                 color_discrete_map={'정상': 'blue', '이상': 'red'})
fig.update_traces(marker=dict(size=8))
fig.show()

---

# Part 1: 기초 - 이상탐지 개념 이해

---

## 1.1 이상탐지 vs 분류

| 구분 | 분류 (Classification) | 이상탐지 (Anomaly Detection) |
|------|----------------------|-----------------------------|
| 레이블 | 모든 클래스에 레이블 필요 | 주로 정상 데이터만 사용 |
| 클래스 비율 | 상대적으로 균형 | 극심한 불균형 (이상 < 1%) |
| 학습 방식 | Supervised | Unsupervised / Semi-supervised |
| 목표 | 클래스 예측 | 이상 여부 판단 |

## 1.2 이상치의 세 가지 유형

### 1) Global Outliers (점 이상)
- 전체 데이터 분포에서 벗어난 점
- 가장 일반적인 형태

### 2) Contextual Outliers (맥락 이상)
- 특정 맥락에서만 이상인 점
- 예: 여름에 난방비 급증

### 3) Collective Outliers (집단 이상)
- 개별로는 정상이지만, 집합적으로 이상
- 예: 연속된 거래 패턴

## 1.3 언제 이상탐지를 사용하나요?

1. **이상 데이터가 매우 적을 때** (< 1%)
2. **이상 데이터에 레이블이 없을 때**
3. **새로운 유형의 이상 탐지가 필요할 때**
4. **정상의 개념을 학습하고 싶을 때**

---

## 1.4 Isolation Forest 원리

### 핵심 아이디어: "이상치는 고립시키기 쉽다"

```
정상 데이터:   ●●●●●●●  → 여러 번 분할해야 고립
이상 데이터:   ○        → 적은 분할로 고립
```

### 작동 방식

1. 랜덤하게 특성과 분할점 선택
2. 재귀적으로 트리 구축
3. **경로 길이(path length)** 측정
4. 경로가 짧을수록 = 이상치 가능성 높음

### 핵심 파라미터

| 파라미터 | 설명 | 권장값 |
|---------|------|--------|
| n_estimators | 트리 개수 | 100 (기본) |
| contamination | 예상 이상치 비율 | 도메인 지식 기반 |
| max_samples | 샘플링 크기 | 'auto' 또는 256 |

In [ ]:
# 기본 Isolation Forest
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.05,  # 5% 이상치 가정
    random_state=42
)

# 학습 및 예측
y_pred_iso = iso_forest.fit_predict(X)

# 이상치 점수 (낮을수록 이상)
scores_iso = iso_forest.decision_function(X)

print("=== Isolation Forest 결과 ===")
print(f"예측된 정상: {np.sum(y_pred_iso == 1)}개")
print(f"예측된 이상: {np.sum(y_pred_iso == -1)}개")

In [ ]:
# 결과 시각화
df['Prediction'] = ['정상' if p == 1 else '이상' for p in y_pred_iso]
df['Anomaly_Score'] = scores_iso

fig = px.scatter(df, x='Feature_1', y='Feature_2', color='Prediction',
                 title='Isolation Forest 이상탐지 결과',
                 color_discrete_map={'정상': 'blue', '이상': 'red'},
                 hover_data=['Anomaly_Score'])
fig.show()

### 💡 실무 예시: decision_function 활용

- **점수 < 0**: 이상치로 판단
- **점수 > 0**: 정상으로 판단
- 점수의 절대값이 클수록 확신도 높음

---

# Part 2: 심화 - 이상탐지 모델 활용

---

## 2.1 contamination 파라미터의 영향

In [ ]:
# contamination에 따른 결과 비교
contamination_values = [0.01, 0.05, 0.10, 0.20]
fig = make_subplots(rows=2, cols=2, subplot_titles=[f'contamination = {c}' for c in contamination_values])

for idx, cont in enumerate(contamination_values):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    iso = IsolationForest(contamination=cont, random_state=42)
    y_pred = iso.fit_predict(X)
    
    # 정상
    fig.add_trace(
        go.Scatter(x=X[y_pred==1, 0], y=X[y_pred==1, 1], mode='markers',
                   marker=dict(color='blue', size=6), name='정상', showlegend=(idx==0)),
        row=row, col=col
    )
    # 이상
    fig.add_trace(
        go.Scatter(x=X[y_pred==-1, 0], y=X[y_pred==-1, 1], mode='markers',
                   marker=dict(color='red', size=8, symbol='x'), name='이상', showlegend=(idx==0)),
        row=row, col=col
    )

fig.update_layout(title='Contamination에 따른 이상탐지 결과 변화', height=600)
fig.show()

### ⚠️ 주의: contamination 설정 가이드

| 상황 | 권장 contamination |
|------|-------------------|
| 도메인 지식 있음 | 실제 이상 비율 사용 |
| 도메인 지식 없음 | 0.01 ~ 0.05 시작 |
| 탐색적 분석 | 여러 값으로 실험 |

---

## 2.2 One-Class SVM

### 핵심 아이디어: "정상 데이터의 경계를 학습"

- 정상 데이터만으로 학습 (Novelty Detection)
- 학습된 경계 밖의 데이터 = 이상

### 핵심 파라미터

| 파라미터 | 설명 | 권장값 |
|---------|------|--------|
| nu | 이상치 비율 상한 | 0.01 ~ 0.1 |
| kernel | 커널 함수 | 'rbf' |
| gamma | RBF 파라미터 | 'scale' |

In [ ]:
# 데이터 스케일링 (One-Class SVM에 필수)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-Class SVM
ocsvm = OneClassSVM(
    kernel='rbf',
    nu=0.05,  # 이상치 비율 상한
    gamma='scale'
)

# 학습 및 예측
y_pred_ocsvm = ocsvm.fit_predict(X_scaled)

print("=== One-Class SVM 결과 ===")
print(f"예측된 정상: {np.sum(y_pred_ocsvm == 1)}개")
print(f"예측된 이상: {np.sum(y_pred_ocsvm == -1)}개")

In [ ]:
# 결과 시각화
df['OCSVM_Prediction'] = ['정상' if p == 1 else '이상' for p in y_pred_ocsvm]

fig = px.scatter(df, x='Feature_1', y='Feature_2', color='OCSVM_Prediction',
                 title='One-Class SVM 이상탐지 결과',
                 color_discrete_map={'정상': 'blue', '이상': 'red'})
fig.show()

---

## 2.3 Isolation Forest vs One-Class SVM 비교

In [ ]:
# 성능 비교
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

results = []

for name, y_pred in [('Isolation Forest', y_pred_iso), ('One-Class SVM', y_pred_ocsvm)]:
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, pos_label=-1)  # 이상을 positive로
    rec = recall_score(y_true, y_pred, pos_label=-1)
    f1 = f1_score(y_true, y_pred, pos_label=-1)
    
    results.append({
        'Model': name,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1': f1
    })

results_df = pd.DataFrame(results)
print("=== 모델 성능 비교 ===")
print(results_df.to_string(index=False))

In [ ]:
# 시각화 비교
fig = make_subplots(rows=1, cols=2, subplot_titles=['Isolation Forest', 'One-Class SVM'])

# Isolation Forest
fig.add_trace(
    go.Scatter(x=X[y_pred_iso==1, 0], y=X[y_pred_iso==1, 1], mode='markers',
               marker=dict(color='blue', size=6), name='정상', showlegend=True),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=X[y_pred_iso==-1, 0], y=X[y_pred_iso==-1, 1], mode='markers',
               marker=dict(color='red', size=8, symbol='x'), name='이상', showlegend=True),
    row=1, col=1
)

# One-Class SVM
fig.add_trace(
    go.Scatter(x=X[y_pred_ocsvm==1, 0], y=X[y_pred_ocsvm==1, 1], mode='markers',
               marker=dict(color='blue', size=6), showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=X[y_pred_ocsvm==-1, 0], y=X[y_pred_ocsvm==-1, 1], mode='markers',
               marker=dict(color='red', size=8, symbol='x'), showlegend=False),
    row=1, col=2
)

fig.update_layout(title='Isolation Forest vs One-Class SVM', height=400)
fig.show()

### 💡 모델 선택 가이드

| 상황 | 추천 모델 |
|------|----------|
| 고차원 데이터 | Isolation Forest |
| 정상 데이터만 있음 | One-Class SVM |
| 빠른 학습 필요 | Isolation Forest |
| 해석 가능성 필요 | Isolation Forest (경로 길이) |

---

## 2.4 실무 활용 사례

### 신용카드 사기 탐지
- 전체 거래의 0.1% 미만이 사기
- Isolation Forest로 이상 거래 탐지

### 제조 공정 불량 탐지
- 센서 데이터에서 비정상 패턴 감지
- One-Class SVM으로 정상 범위 학습

### 서버 로그 이상 감지
- 비정상적인 트래픽/응답 시간 탐지
- Isolation Forest로 실시간 모니터링

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 기본 Isolation Forest ⭐

**문제**: Isolation Forest를 학습하고 예측된 이상치 개수를 출력하세요.

```python
from sklearn.ensemble import IsolationForest
# X는 이미 준비됨
```

In [ ]:
# 여기에 코드를 작성하세요


### Q2. contamination 설정 ⭐

**문제**: contamination=0.10인 Isolation Forest를 학습하고 이상치 비율을 확인하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 이상치 점수 확인 ⭐⭐

**문제**: decision_function으로 이상치 점수를 계산하고, 점수가 가장 낮은 5개 데이터를 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 이상치 시각화 ⭐⭐

**문제**: Isolation Forest 결과를 Plotly 산점도로 시각화하세요. (정상/이상 색상 구분)

In [ ]:
# 여기에 코드를 작성하세요


### Q5. One-Class SVM 기본 ⭐⭐

**문제**: One-Class SVM을 학습하고 예측 결과를 출력하세요. (스케일링 포함)

In [ ]:
# 여기에 코드를 작성하세요


### Q6. nu 파라미터 실험 ⭐⭐⭐

**문제**: nu=0.01, 0.05, 0.10 세 가지 값으로 One-Class SVM을 학습하고 이상치 개수를 비교하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 성능 평가 ⭐⭐⭐

**문제**: y_true와 예측 결과를 비교하여 Accuracy, Precision, Recall을 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q8. contamination 최적화 ⭐⭐⭐⭐

**문제**: contamination을 0.01~0.20까지 변화시키며 F1 스코어가 가장 높은 값을 찾으세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 모델 비교 ⭐⭐⭐⭐

**문제**: Isolation Forest와 One-Class SVM의 성능을 DataFrame으로 비교하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 종합 시각화 ⭐⭐⭐⭐⭐

**문제**: 두 모델의 결과를 make_subplots로 나란히 시각화하고, 실제 레이블과 비교하세요.

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 이상탐지 기초 요약

| 개념 | 설명 | 핵심 포인트 |
|------|------|------------|
| 이상탐지 | 비정상 데이터 찾기 | 극심한 불균형 문제 해결 |
| Global Outlier | 전체 분포에서 이상 | 가장 일반적 |
| Isolation Forest | 고립 기반 탐지 | 경로가 짧으면 이상 |
| contamination | 예상 이상치 비율 | 도메인 지식 필요 |

### Part 2: 이상탐지 활용 요약

| 모델 | 특징 | 추천 상황 |
|------|------|----------|
| Isolation Forest | 빠름, 해석 가능 | 고차원, 대용량 |
| One-Class SVM | 정상 경계 학습 | 정상 데이터만 있을 때 |

### 💡 실무 팁

1. **contamination 신중히**: 도메인 지식 기반으로 설정
2. **스케일링 확인**: One-Class SVM은 스케일링 필수
3. **decision_function 활용**: 점수로 우선순위화 가능
4. **F1 스코어 중시**: 불균형 데이터에서 Accuracy는 무의미
5. **실시간 모니터링**: Isolation Forest가 빠르고 효율적